<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 15.3 MB/s 
     |████████████████████████████████| 182 kB 56.2 MB/s 
     |████████████████████████████████| 168 kB 67.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 68.0 MB/s 
     |████████████████████████████████| 166 kB 74.7 MB/s 
     |████████████████████████████████| 166 kB 75.4 MB/s 
     |████████████████████████████████| 162 kB 54.5 MB/s 
     |████████████████████████████████| 162 kB 57.6 MB/s 
     |████████████████████████████████| 158 kB 32.0 MB/s 
     |████████████████████████████████| 157 kB 58.5 MB/s 
     |████████████████████████████████| 157 kB 48.2 MB/s 
     |████████████████████████████████| 157 kB 62.4 MB/s 
     |████████████████████████████████| 157 kB 58.9 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 71.0 MB/s 
     |████████████████████████████████| 157 kB 79.2 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [7]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [14]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_xception=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  xception,
  # layers.Flatten(),
  # layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
xception.trainable=False
model_xception.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 20,871,725
Trainable params: 10,245
Non-trainable params: 20,861,480
_________________________________________________________________


In [15]:
model_xception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [16]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Xception_with_avgpooling",
)

In [17]:
%%time
epochs=40
history = model_xception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.1465 - accuracy: 0.6037

wandb: Adding directory to artifact (/content/wandb/run-20221128_091736-2xjqd121/files/model-best)... Done. 0.8s


22/22 [==============================] - 28s 1s/step - loss: 1.1465 - accuracy: 0.6037 - val_loss: 0.7786 - val_accuracy: 0.7886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.6223 - accuracy: 0.8523

wandb: Adding directory to artifact (/content/wandb/run-20221128_091736-2xjqd121/files/model-best)... Done. 0.8s


22/22 [==============================] - 24s 1s/step - loss: 0.6223 - accuracy: 0.8523 - val_loss: 0.5211 - val_accuracy: 0.9086
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 0.4249 - accuracy: 0.9205

...

22/22 [==============================] - 25s 1s/step - loss: 0.0262 - accuracy: 0.9986 - val_loss: 0.0867 - val_accuracy: 0.9657
Epoch 38/40
22/22 [==============================] - 5s 205ms/step - loss: 0.0255 - accuracy: 1.0000 - val_loss: 0.0870 - val_accuracy: 0.9657
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0264 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221128_091736-2xjqd121/files/model-best)... Done. 0.9s


22/22 [==============================] - 25s 1s/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 0.0846 - val_accuracy: 0.9657
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.0253 - accuracy: 0.9986

wandb: Adding directory to artifact (/content/wandb/run-20221128_091736-2xjqd121/files/model-best)... Done. 0.9s


22/22 [==============================] - 25s 1s/step - loss: 0.0253 - accuracy: 0.9986 - val_loss: 0.0837 - val_accuracy: 0.9714
CPU times: user 13min 29s, sys: 57 s, total: 14min 26s
Wall time: 20min 48s


In [18]:
wandb.alert(
    title = 'Run finished',
    text = '40 epochs done'
)

In [19]:
wandb.finish()

accuracy,▁▅▇▇▇▇▇█████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
val_loss,█▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99858
best_epoch,39
best_val_loss,0.08366
epoch,39
loss,0.02529
val_accuracy,0.97143
